In [2]:
from pdfminer.high_level import extract_text
import cohere
import spacy
import re
from IPython.display import display, Markdown

co = cohere.Client("YOUImGXa6Wg0XgfsFdM0c2iIFOX3fM8UDMKzdGYZ")

# Extracting text from pdf as is
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

# Cleaning up the text for better processing in the later stages
def clean_text(text):
    text = re.sub(r'\n+', '\n', text)   # Remove newlines
    text = re.sub(r'\s+', ' ', text)    # Remove multiple spaces
    text = re.sub(r'Page \d+ of \d+', '', text) # Remove page numbers
    text = text.replace('•', ' ')   # Replace bullets with space.
    text = re.sub(r'\+46-\d{9}', lambda x: x.group().replace('-', ''), text) # Remove hyphens or spaces in phone number 
    text = text.strip() # Remove redundant spaces
    return text

# Main part of the script - Taking out important data
nlp = spacy.load("en_core_web_lg")
EMAIL_PATTERN = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
PHONE_PATTERN = r'(?:(?:\+|00)(?:\d{1,3}))?[-.\s]?(?:\(?\d{1,4}\)?)?[-.\s]?\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9}'

# Skill finder
JD = open("jobDesc.txt",'r').read()
response = co.chat(
	message = "Following is a job description of a role I want to apply to:\n" + JD + "\n\nImagine you are a seasoned hiring manager. List the main keywords you would put in this job description that candidates need to fulfill. Give those keywords a weighing score from 1 to 10 in order of relevance. Feel free to merge similar keywords and aggregate their score. Output the main keywords and their scores as a dictionary, only keep the dictionary, no extra word"
)
skills_list = response.text

def extract_info(text):
    # Process text with spaCy
    doc = nlp(text)
    extracted_data = {
        "names": [],
        "emails": [],
        "phones": []
    }

    # Name
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            if not extracted_data["names"]:
                extracted_data["names"].append(ent.text)

    # Phone number and email
    extracted_data["emails"] = re.findall(EMAIL_PATTERN, text)
    extracted_data["phones"] = max(re.findall(PHONE_PATTERN, text), key=len)
    # The phone number extraction also extracts dates from the later sections.
    # Thus, max() is used with the length of string being the primary keyword.
    # Hopefully it works in most, if not all, cases.    

    return extracted_data

def extract_skill(text, reqs):
    response = co.chat(
        message = "Following is my resume:\n" + text + "\n<end of resume>\nI am applying for a job which has the keywords and their respective percieved relevance in this dictionary:\n" + str(reqs) + "\nPlease provide a score out of 100 indicating how well my skills and projects match the job requirements, without any sugarcoating. Additionally, offer recommendations for changes I can make to my resume to maximize my chances of an interview, excluding any suggestions related to work experience, roles, responsibilities, proofreading, cover letter tailoring, or English proficiency. Use markdown for formatting."
    )
    return response.text

filename = "AbhishekKolekar_Resume.pdf"
textFromResume = extract_text_from_pdf(filename)
infoFromResume = extract_info(textFromResume)
skills_match = extract_skill(textFromResume, skills_list)
display(Markdown(skills_match))

# Resume Evaluation:

**Score: 85/100**

Your resume demonstrates a strong fit for the job, particularly in the areas of automotive engineering, simulation, and energy efficiency. Here are some observations and suggestions to enhance your application:

## Skills and Projects:
- **CAE Analyst (10/10):** Your experience with various software tools like MATLAB, IPG CarMaker, and VI-CarRealTime aligns well with CAE analysis. Highlighting specific CAE-related tasks, such as designing models, simulations, and optimizations, could further emphasize your suitability.

- **Energy Efficiency (9/10):** The projects on tire warm-up and rolling resistance, as well as drivetrain optimization, showcase your expertise in this area. Ensure these projects are prominently featured to capture the recruiter's attention.

- **Electric Vehicles (9/10):** The Lap-Time Simulation project for electric vehicles is highly relevant. Emphasize the energy consumption estimation aspect and its potential impact on EV design.

- **Mathematical Modeling (8/10):** Your work on various models (Bicycle Model, Thermal-Schuring model) and optimizations demonstrates this skill. Consider adding a brief description of the models you've worked with to provide more context.

- **Optimization (8/10):** The use of Reinforcement Learning for optimization is impressive. Elaborate on the impact of your optimizations on vehicle performance and efficiency.

- **Simulation (8/10):** You have extensive simulation experience, as evidenced by your projects. List the simulation software you are proficient in under the 'Skillset' section.

- **Vehicle System Physics (7/10):** Your understanding of vehicle dynamics is evident, but adding more details about your coursework in Vehicle Dynamics and Control Systems could strengthen this aspect.

## Recommendations for Resume Enhancement:

**1. Technical Skills Section:**
   - Expand the 'Skillset' section to include sub-sections:
     - **Software Proficiency:** MATLAB/Simulink, Python, C++, SOLIDWORKS, CATIA V5, ANSYS Mechanical, IPG CarMaker, VI-CarRealTime.
     - **Tools & Protocols:** CAN Communication, HIL/SIL Testing, MDA, INCA, Statistical Data Analysis.
   - This structured format ensures recruiters quickly grasp your technical expertise.

**2. Project Descriptions:**
   - Provide concise bullet points for each project, highlighting:
     - Project objective.
     **Methodology/Tools used.**
     - Key achievements or results.
     - For instance: "Developed a MATLAB-based LTS for EVs, improving battery design efficiency by 15%."

**3. Education:**
   - Briefly describe a relevant project or thesis for each degree, e.g., "MSc Thesis: Tire Warmup and Rolling Resistance Modeling."

**4. Certifications:**
   - List certifications related to automotive or engineering software, e.g., SOLIDWORKS, MATLAB, or any CAE-specific certifications.

**5. Languages:**
   - Specify language proficiency levels using a recognized framework (e.g., CEFR) to provide a clear indication of your English and Swedish language skills.

**6. Additional Keywords:**
   - Incorporate keywords from the job description into your resume, especially in project descriptions and skill highlights. This ensures that your resume aligns closely with the job requirements.

Remember, the goal is to present your skills and experiences in a way that is easily digestible for recruiters, showcasing your potential value to the company. Good luck with your job application!